# Regression Analysis

#### Evan Yathon

This notebook is intended to be run with papermill from the project root.

The purpose of this notebook is to run a logistic regression analysis on the cleaned reviews data.  Trying to identify contributing factors to a customer recommending an airline or not given that they have submitted a review is the central theme.  Feature importance, coefficient estimates and confidence intervals are pulled from models after some initial cleanup is performed. 

Usage:

`papermill src/ipynbs/regression_analysis.ipynb src/regression_analysis_ran.ipynb -p load_path data/topic_modeling_gw_reviews.csv`

In [1]:
load_path = "../../data/topic_modeling_gw_reviews.csv"

In [2]:
# Parameters
load_path = "data/topic_modeling_gw_reviews.csv"


In [3]:
# load packages

# utils
import pandas as pd
import sys
import numpy as np
import time

# regression
from sklearn.linear_model import LogisticRegression

# regression utils
# custom model preparation script
sys.path.append("../../src")
sys.path.append("./src")
from PrepareForModel import * # dummy encoding 
from bootstrap_skmodel import * # regression coefficient bootstrapping


In [4]:
reviews = pd.read_csv(load_path, parse_dates = ["date_of_review", "date_flown"])

### Regression Prep

Before running the regression analysis, a few things need to be done.

- Remove dates after the transfer to Eurowings
- Drop columns that have large amount of `NA` values or don't make sense to include in the analysis
- Deal with `NA` values
- Encode categorical columns

Remove dates after `8th February 2016` as Germanwings was transferred to Eurowings (as found in the EDA notebook).

In [5]:
reviews_regr = reviews[reviews["date_of_review"] < "2016-02-08"]

Check column types and percentage of NA values

In [6]:
# column types, na values
pd.DataFrame({
    "data_type" : reviews_regr.dtypes.values,
    "precent_na" : (reviews_regr.isna().sum()/reviews_regr.shape[0])*100
})

,data_type,precent_na
title,object,0.000000
review_value,float64,9.420290
n_user_reviews,object,92.753623
reviewer_name,object,0.000000
reviewer_country,object,9.420290
date_of_review,datetime64[ns],0.000000
review_text,object,0.000000
aircraft,object,91.304348
traveller_type,object,82.608696
seat_type,object,10.144928


`object` types are categorical/strings.  Below in a table documenting which columns will be dropped and the reasoning behind each.

| Column Name                   	| Reason to Drop                                                  	|
|-------------------------------	|-----------------------------------------------------------------	|
| title                         	| Information extracted via Topic Modeling                        	|
| n_user_reviews                	| Large amount of missing values                                  	|
| reviewer_name                 	| Not relevant for regression analysis                            	|
| reviewer_country              	| Too many categories for small amount of data; EDA backs this up 	|
| date_of_review                	| imbalanced dataset over time                                    	|
| review_text                   	| Information extracted via Topic Modeling                        	|
| aircraft                      	| Large amount of missing values                                  	|
| traveller_type                	| Large amount of missing values                                  	|
| route                         	| Large amount of missing values                                  	|
| date_flown                    	| Large amount of missing values                                  	|
| ground_service_rating         	| Large amount of missing values                                  	|
| inflight_entertainment_rating 	| Large amount of missing values                                  	|
| clean_review_text             	| Information extracted via Topic Modeling                        	|
| clean_title                   	| Information extracted via Topic Modeling                        	|

In [7]:
reviews_regr.columns

Index(['title', 'review_value', 'n_user_reviews', 'reviewer_name',
       'reviewer_country', 'date_of_review', 'review_text', 'aircraft',
       'traveller_type', 'seat_type', 'route', 'date_flown',
       'seat_comfort_rating', 'cabin_staff_service_rating',
       'food_and_beverages_rating', 'inflight_entertainment_rating',
       'ground_service_rating', 'value_for_money_rating', 'recommendation',
       'clean_review_text', 'clean_title', 'review_luggage_seats',
       'review_time_delays', 'review_food_bev_crew', 'title_money_value',
       'title_staff_delays'],
      dtype='object')

In [8]:
drop_cols = ['title', 'n_user_reviews', 'reviewer_name',
             'reviewer_country', 'date_of_review', 'review_text', 
             'aircraft', 'traveller_type', 'route', 'date_flown',
             'ground_service_rating', 'inflight_entertainment_rating',
             'clean_review_text', 'clean_title']

In [9]:
# drop unwanted columns
reviews_regr = reviews_regr.drop(drop_cols, axis = 1)

After dropping, review the na values and column types again

In [10]:
# column types, na values
pd.DataFrame({
    "data_type" : reviews_regr.dtypes.values,
    "precent_na" : (reviews_regr.isna().sum()/reviews_regr.shape[0])*100
})

,data_type,precent_na
review_value,float64,9.420290
seat_type,object,10.144928
seat_comfort_rating,float64,34.057971
cabin_staff_service_rating,float64,34.057971
food_and_beverages_rating,float64,36.231884
value_for_money_rating,float64,9.420290
recommendation,object,0.000000
review_luggage_seats,float64,0.000000
review_time_delays,float64,0.000000
review_food_bev_crew,float64,0.000000


In [11]:
# levels of the seat_type variable and counts
reviews_regr.seat_type.value_counts()

Economy Class     122
First Class         1
Business Class      1
Name: seat_type, dtype: int64

Seat comfort, cabin staff service and food and beverages still have a fair amount missing, I might need to remove them from the analysis later, or perform two separate analyses.

`seat_type` and `recommendation` need to be encoded.  To do that, I'll use some code that I had developed prior.

`PrepareForModel` dummy encodes categorical variables in a given dataframe.  There also exists an option to effect code the categorical variables, but for this option I will keep effect coding.

The reason for effect coding is that I plan to have `Economy Class` as the reference variable, and perform a reference treatment effect to see if moving to business or first class would increase the chance of a recommendation.  An issue with this is that there is only one business class and one first class seat in the dataset.

### Analysis

In interest of time, I will also drop the seat comfort, cabin staff service and food and beverages rating.  The reasoning behind this is to preserve as many samples as possible in order to increase certainty in the regression analysis.

In the analysis I will do three things:
1. Obtain relative feature importances via L1 regularization with varying strengths
2. Estimate regression coefficients
3. Obtain a 95% CI via bootstrapping


`scikit-learn` was chosen for this task over another module such as `statsmodels`.  `statsmodels` would provide a CLT estimate of the coefficients, giving a CI by default.  But `scikit-learn` allows regression to be performed with L1 regularization, my chosen method for feature importance.

In [12]:
# remove columns with large amounts of NA values, then drop all rows with NA values
review_regr = reviews_regr.drop(
    ["seat_comfort_rating", "cabin_staff_service_rating", "food_and_beverages_rating"], axis = 1).dropna()
prep = PrepareForModel(review_regr)
review_regr_enc = prep.make_dummy_df({"seat_type" : "Economy Class", "recommendation" : "no"}, add_intercept = False)

In [13]:
# split into X and Y
# don't worry about test/train splits as we are only concerned about the relationship
# between variables and the response rather than predicting anything
X = review_regr_enc.drop(["recommendation_yes"], axis = 1)
y = review_regr_enc["recommendation_yes"]

### Feature Importances

In [14]:
# specify L1 regularization
logit_l1_params = {"penalty" : "l1",
                "fit_intercept" : True,
                "random_state" : 42,
                "solver" : "liblinear", #required for accurate coefficients
                "max_iter" : 100}

In [15]:
# run through varying regularization strength values
# note that C is inverse regularization strength
C_vals = 6**np.linspace(-2, 8, 11)

# create empty dataframe to fill in
C_df = pd.DataFrame()


for count, C in enumerate(C_vals):
    
    # set up the column name
    col = "C_" + str(round(C,3))
    
    # create the regression object with specified reg. strength
    logit_l1 = LogisticRegression(**logit_l1_params, C = C)
    logit_l1.fit(X,y)
    
    # assign coeffients to the correct place in the matrix
    C_df[col] = logit_l1.coef_.flatten()

C_df.index = X.columns

Feature importance in this case is defined as features that still have a coefficient at high regularization strength values.  If I look at the order in which feature's coefficients become zero with increasing regularization strength, this should give relative feature importance order.

In [16]:
# view the resulting dataframe
C_df

,C_0.028,C_0.167,C_1.0,C_6.0,C_36.0,C_216.0,C_1296.0,C_7776.0,C_46656.0,C_279936.0,C_1679616.0
review_value,0.167544,0.592905,1.260665,1.915156,2.290198,2.387994,2.412487,2.410818,2.416485,2.412993,2.412995
value_for_money_rating,0.000000,0.000000,0.287325,0.586043,0.692446,0.715612,0.720055,0.721591,0.720965,0.716558,0.716553
review_luggage_seats,0.000000,0.000000,0.000000,0.000000,-0.147750,-4.348466,-6.613777,-5.198756,-8.032637,-15.500214,-15.516508
review_time_delays,0.000000,0.000000,0.000000,0.000000,0.000000,-4.114008,-6.371325,-4.956095,-7.789375,-15.263695,-15.279997
review_food_bev_crew,0.000000,0.000000,0.000000,-0.019486,-0.436438,-4.686347,-6.961945,-5.541425,-8.387801,-15.881671,-15.898028
title_money_value,0.000000,0.000000,0.000000,-3.046914,-6.070140,-6.987636,-6.538638,-7.572138,-5.731944,-0.787135,-0.773093
title_staff_delays,0.000000,0.000000,0.000000,0.000000,0.000000,-0.241771,0.190083,-0.803397,1.064521,6.007047,6.021272
seat_type_Business Class,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.721005
seat_type_First Class,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.483965,-2.294960,-4.086723


In [17]:
# extract the order of feature importance

feature_importance_order = (C_df != 0).sum(axis = 1).sort_values(ascending = False).index.tolist()

In [18]:
# view the feature importance order
feature_importance_order

['review_value',
 'value_for_money_rating',
 'title_money_value',
 'review_food_bev_crew',
 'review_luggage_seats',
 'title_staff_delays',
 'review_time_delays',
 'seat_type_First Class',
 'seat_type_Business Class']

### Regression Coefficients

In [19]:
# get regression coefficients
logit_params = {"penalty" : "none",
                "C" : 1.0,
                "fit_intercept" : True,
                "random_state" : 42,
                "solver" : "newton-cg", #required for accurate coefficients
                "max_iter" : 100}

In [20]:
# initialize the model
logit = LogisticRegression(**logit_params)

In [21]:
# fit the model, extract coefficients
logit.fit(X,y)
logit_coefs = logit.coef_.flatten()

In [22]:
# start to build the final dataframe containing results
review_results_df = pd.DataFrame({
    "variable_name" : X.columns,
    "coefficient_estimate" : logit_coefs,
    "feature_importance_value" : (C_df == 0).sum(axis = 1)
})

### Bootstrapped Coefficients for Confidence Intervals

Bootstrapped coefficients will be used to derive confidence intervals and pseudo p-values to get an idea of how good each coefficient estimate is.

In [23]:
start = time.time()
boot_coefs = parallel_bootstrap(skmodel = LogisticRegression, 
                                skmodel_args = logit_params,
                                X = X,
                                y = review_regr_enc[["recommendation_yes"]], # requires a pd.DataFrame rather than a np array
                                n_bootstraps = 5000)

print("Bootstrapping took {}".format(time.time() - start))

Bootstrapping took 54.0082790851593


In [24]:
# extract the 95% confidence interval from the bootstrapped coefficients
review_results_df["quantile2.5"] = np.quantile(boot_coefs, 0.025, axis = 0)
review_results_df["quantile97.5"] = np.quantile(boot_coefs, 0.975, axis = 0)

In [25]:
review_results_df.sort_values(by = "feature_importance_value")

,variable_name,coefficient_estimate,feature_importance_value,quantile2.5,quantile97.5
review_value,review_value,2.428162,0,1.789010,2894.348496
value_for_money_rating,value_for_money_rating,0.692510,2,-1.339488,36.609167
review_food_bev_crew,review_food_bev_crew,-124.852794,3,-4012.241498,53.032617
title_money_value,title_money_value,33.934106,3,-4630.907210,875.150363
review_luggage_seats,review_luggage_seats,-124.231626,4,-4039.637143,55.119078
review_time_delays,review_time_delays,-124.051118,5,-2375.644515,70.291347
title_staff_delays,title_staff_delays,40.243784,5,-2489.441351,1807.150002
seat_type_First Class,seat_type_First Class,-2.915885,8,-5.328878,0.000000
seat_type_Business Class,seat_type_Business Class,0.509681,10,0.000000,3.733909


It turns out that the coefficients vary wildly.  The only coefficients that somewhat make sense are the two least important ones, first class and business class seat types.  We know that they only had one review for each one.

The feature importance order makes sense, but I don't think we can put any stock into the coefficients.  The reason I think the feature importance order is somewhat reliable is because it is backed up by what was seen in the EDA notebook.

Small sample size is the culprit here, no coefficient has enough power behind it to make a solid conclusion.